In [3]:
def get_data(endpoint, params):
    
    URL = "https://v3.football.api-sports.io/"
    headers = {
	'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': "fb2140228973d644db847895c454c22b"
    }
    
    response = requests.get(
        URL+endpoint,
        headers = headers,
        params = params
    )
    if response.status_code == 200:
            
        remaining = response.headers.get("x-ratelimit-requests-remaining")
        data = response.json()
        print(f"requests before reaching limit {remaining}")

    else:
        print(f"Error {response.status_code}, {response.text}")

    return data, remaining                                                                     

In [4]:
def encode_data(data_dict, parent_key = '', sep= '_'):
    encoded = []
    for key, val in data_dict.items():
        new_key = f'{parent_key}{sep}{key}' if parent_key else key
        if isinstance(val, dict):
            encoded.extend(encode_data(val, new_key, sep=sep).items())
        elif isinstance(val, list):
            if val:
                if all(isinstance(i, dict) for i in val):
                    for k, v in enumerate(val):
                        v_key = f'{new_key}{sep}{k}'
                        encoded.extend(encode_data(v, v_key, sep=sep).items())
                else:
                    encoded.append((new_key, val))
            else:
                encoded.append((new_key, []))
        else:
            encoded.append((new_key, val))
    return dict(encoded)

In [5]:
def data_to_sql(table_name, df, conflict_columns, update_columns = None):
    conn = None
    cur = None
    params = {
    'host': 'localhost',
    'database': 'preds',
    'user': 'postgres',
    'password': 'pass',
    'port': '5432'
}
    try:
        # Establish the connection
        conn = psycopg2.connect(**params)
        cur = conn.cursor()

        if not update_columns:
            update_columns = [col for col in df.columns if col not in conflict_columns]
        #insert data into tables
        if len(conflict_columns) == 0:
            insert_query = """
                INSERT INTO {} ({})
                VALUES ({})
            """.format(table_name, ','.join(df.columns), ','.join(['%s']*len(df.columns)))
        else:
            if update_columns:
                update_set = ', '.join([f"{col} = EXCLUDED.{col}" for col in update_columns])
                insert_query = """
                    INSERT INTO {} ({})
                    VALUES ({})
                    ON CONFLICT ({}) DO UPDATE SET {}
                """.format(table_name, ','.join(df.columns), ','.join(['%s']*len(df.columns)), ','.join(conflict_columns), update_set)

            else:
                insert_query = """
                    INSERT INTO {} ({})
                    VALUES ({})
                    ON CONFLICT ({}) DO NOTHING
                """.format(table_name, ','.join(df.columns), ','.join(['%s']*len(df.columns)), ','.join(conflict_columns))
        cur.executemany(insert_query, df.values.tolist())
        print(f'table {table_name} updated')
        
        # Commit the changes
        conn.commit()
        
    except Exception as e:
        print(f"Error: {e}")
        if conn is not None:
            conn.rollback()
    finally:
        if conn is not None:
            # Close the cursor and connection
            cur.close()
        if cur is not None:
            conn.close()

In [36]:
from datetime import date, timedelta
import datetime
import pandas as pd
import psycopg2
import json

today = date.today()
end = today - timedelta(days=7)
es = pd.read_csv('../data/european_seasons.csv')
s = pd.read_csv('../data/seasons.csv')
seasons = pd.merge(es,s[['league_id','year','start','end']], on = ['league_id', 'year'])
seasons['start'] = pd.to_datetime(seasons['start']).dt.date
seasons['end'] = pd.to_datetime(seasons['end']).dt.date

In [37]:
current_seasons = seasons[(seasons['start']<today)&(seasons['end']>end)][['league_id', 'year']]
leagues = list(current_seasons['league_id'].unique())
seasons = list(current_seasons['year'].unique())
leagues = [int(league) for league in leagues]
seasons = [str(season) for season in seasons]

In [39]:
import requests

remaining = 100
done = False
fix_data = []
to_get = current_seasons.values.tolist()

path = '../data/last.json'
with open(path, 'r') as file:
    last = json.load(file)
if len(last)>0:
    if not last[2]:
        ind = to_get.index([last[0], last[1]])
        to_get = to_get[ind:]

while (remaining > 0)&(done == False):
    params = {'league' : to_get[0][0], 'season': to_get[0][1]}
    data, remaining = get_data('fixtures', params)
    remaining = int(remaining)
    if len(data['response'])>0:
        fix_data.extend(encode_data(fix) for fix in data['response'])
    
    to_get.pop(0)
    if len(to_get) == 0:
        done = True

if remaining <= 0 and not done:
    last = [to_get[0][0],to_get[0][1], done]
    path = '../data/last.json'
    with open(path, 'w') as file:
        json.dump(last, file)

requests before reaching limit 7201
requests before reaching limit 7200
requests before reaching limit 7199
requests before reaching limit 7198
requests before reaching limit 7197
requests before reaching limit 7196
requests before reaching limit 7195
requests before reaching limit 7194
requests before reaching limit 7193
requests before reaching limit 7192
requests before reaching limit 7191
requests before reaching limit 7190
requests before reaching limit 7189
requests before reaching limit 7188
requests before reaching limit 7187
requests before reaching limit 7186
requests before reaching limit 7185
requests before reaching limit 7184
requests before reaching limit 7183
requests before reaching limit 7182
requests before reaching limit 7181
requests before reaching limit 7180
requests before reaching limit 7179
requests before reaching limit 7178
requests before reaching limit 7177
requests before reaching limit 7176
requests before reaching limit 7175
requests before reaching lim

In [42]:
df = pd.DataFrame(fix_data)

In [43]:
df

,fixture_id,fixture_referee,fixture_timezone,fixture_date,fixture_timestamp,fixture_periods_first,fixture_periods_second,fixture_venue_id,fixture_venue_name,fixture_venue_city,...,goals_home,goals_away,score_halftime_home,score_halftime_away,score_fulltime_home,score_fulltime_away,score_extratime_home,score_extratime_away,score_penalty_home,score_penalty_away
0,1163460,R. Toom,UTC,2024-03-02T15:00:00+00:00,1709391600,1.709392e+09,1.709395e+09,18575.0,Laagri kunstmuru,Tallinn,...,2.0,2.0,1.0,0.0,2.0,2.0,NaN,NaN,NaN,NaN
1,1163461,D. Nõmm,UTC,2024-03-02T15:00:00+00:00,1709391600,1.709392e+09,1.709395e+09,NaN,Viljandi Männimäe Jalgpallihall,Viljandi,...,1.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN
2,1163462,J. Jaanovits,UTC,2024-03-02T17:00:00+00:00,1709398800,1.709399e+09,1.709402e+09,10593.0,Sportland Arena,Tallinn,...,2.0,0.0,2.0,0.0,2.0,0.0,NaN,NaN,NaN,NaN
3,1163463,V. Karnatsevitš,UTC,2024-03-03T15:00:00+00:00,1709478000,1.709478e+09,1.709482e+09,11599.0,Tabasalu Staadion,Tabasalu,...,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
4,1163464,R. Maalinn,UTC,2024-03-04T16:00:00+00:00,1709568000,1.709568e+09,1.709572e+09,10593.0,Sportland Arena,Tallinn,...,1.0,3.0,1.0,0.0,1.0,3.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10788,1199842,None,UTC,2024-12-06T12:00:00+00:00,1733486400,NaN,NaN,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10789,1199843,None,UTC,2024-12-06T12:00:00+00:00,1733486400,NaN,NaN,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10790,1199844,None,UTC,2024-12-06T12:00:00+00:00,1733486400,NaN,NaN,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10791,1199845,None,UTC,2024-12-06T12:00:00+00:00,1733486400,NaN,NaN,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
update_cols = list(df.columns)
update_cols.pop(0)

'fixture_id'

In [49]:
update_cols

['fixture_referee',
 'fixture_timezone',
 'fixture_date',
 'fixture_timestamp',
 'fixture_periods_first',
 'fixture_periods_second',
 'fixture_venue_id',
 'fixture_venue_name',
 'fixture_venue_city',
 'fixture_status_long',
 'fixture_status_short',
 'fixture_status_elapsed',
 'league_id',
 'league_name',
 'league_country',
 'league_logo',
 'league_flag',
 'league_season',
 'league_round',
 'teams_home_id',
 'teams_home_name',
 'teams_home_logo',
 'teams_home_winner',
 'teams_away_id',
 'teams_away_name',
 'teams_away_logo',
 'teams_away_winner',
 'goals_home',
 'goals_away',
 'score_halftime_home',
 'score_halftime_away',
 'score_fulltime_home',
 'score_fulltime_away',
 'score_extratime_home',
 'score_extratime_away',
 'score_penalty_home',
 'score_penalty_away']

In [ ]:
if len(fix_data) > 0:
    send_to_sql('fixtures', df, 'fixture_id')